In [8]:
import faiss
import os
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
# from langchain.prompts import PromptTemplate
# from langchain.prompts import ChatPromptTemplate
# from langchain.chains import LLMChain

In [9]:
embeddings=HuggingFaceEmbeddings(model_name="BAAI/bge-large-en")

/home/lahirum/Documents/Mirada/RnD/FastApi/test2/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0


In [19]:
#load pdf files
file_path = "./uploaded_pdfs/sample.pdf"
if os.path.exists(file_path):
    loader = PyPDFLoader(file_path)
    print("file loaded successfully")
else:
    raise FileNotFoundError(f"File not found: {file_path}")

file loaded successfully


In [4]:
#load text
from langchain_community.document_loaders import TextLoader

file_path = "./uploaded_pdfs/sample.txt"
if os.path.exists(file_path):
    loader = TextLoader(file_path)
    print("file loaded successfully")
else:
    raise FileNotFoundError(f"File not found: {file_path}")

file loaded successfully


In [66]:
# #load URLs
# url = ""
# try:
#     loader=WebBaseLoader(url)
#     docs.extend(loader.load())

# except Exception as e:
#     print(f"Error loading from {url}:{e}")

In [20]:
#Load documents
documents = loader.load()

In [ ]:
documents

[Document(metadata={'source': './uploaded_pdfs/sample.txt'}, page_content="Cipherpoint Limited (Formerly known as Covata Limited) Appendix 4E Preliminary final report\n\n## 1. Company details\n\nName of entity:\n\nCipherpoint Limited (Formerly known as Covata Limited)\n\nABN:\n\n61 120 658 497\n\nReporting period:\n\nFor the year ended 31 March 2020\n\nPrevious period:\n\nFor the nine-month period ended 31 March 2019\n\n## 2. Results for announcement to the market\n\nIn  the  current  period,  Cipherpoint  Limited  ('the  Group')  has  adopted  all  of  the  new  and  revised  Standards  and Interpretations issued by the Australian Accounting Standards Board that are relevant to its operations and effective for annual reporting periods commencing on or after 1 January 2019.\n\nIn 2019, the Group changed its financial year from 30 June to 31 March. The financial statements have been prepared for the year ended 31 March 2020, while the comparative accounting period is for the 9 months en

In [9]:
#Split documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=5000,   #1000   #2000 #5000
    chunk_overlap=500,
    separators = ["##", "###", "\n\n", "\n", " ", ""]
)
document_chunks = text_splitter.split_documents(documents)
print(f"document splitted into {len(document_chunks)}")

document splitted into 50


In [ ]:
# Create the FAISS vector store
vector_store = FAISS.from_documents(document_chunks, embeddings)

In [ ]:
#save vector store locally
vector_store.save_local(f"vector_store")

In [10]:
#load vector store
vector_store=FAISS.load_local("vector_store",embeddings,allow_dangerous_deserialization=True)

In [97]:
#   # Create vector index
# index, texts = create_vector_index(document_chunks, embedder)

In [98]:
#create a retriever
# retriever= vector_store.as_retriever()
# retriever = vector_store.as_retriever(
#     search_kwargs={"k":1}
# )

Retrieval

chunk_size=2000

In [11]:
#Maximum marginal relevance retrieval
retriever = vector_store.as_retriever(
    search_type="mmr"
)

In [ ]:
ans=retriever.invoke("What is the total face value of the convertible note?")  #The total face value of the convertible note was $1,400,000. 
print(ans)

In [24]:
#Similarity score threshold retrieval
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
ans=retriever.invoke("What is the total face value of the convertible note?")
print(ans)

[Document(id='af9d2740-0298-42e5-a4d0-8c34aaa60ec2', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                

In [6]:
#Specifying top k
retriever = vector_store.as_retriever(search_kwargs={"k": 1})
ans=retriever.invoke("What is the total face value of the convertible note?")
print(ans)

[Document(id='3e138805-138c-4410-b78c-2d1f75ee707e', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                

chunk_size = 5000

In [16]:
#Maximum marginal relevance retrieval
retriever = vector_store.as_retriever(
    search_type="mmr"
)
docs=retriever.invoke("What is the total face value of the convertible note?")
print(docs)

[Document(id='3e138805-138c-4410-b78c-2d1f75ee707e', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                

In [7]:
#Similarity score threshold retrieval
retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.5}
)
ans=retriever.invoke("What is the total face value of the convertible note?")
print(ans)

[Document(id='3e138805-138c-4410-b78c-2d1f75ee707e', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                

In [ ]:
#Specifying top k
retriever = vector_store.as_retriever(search_kwargs={"k": 1})


[Document(id='af9d2740-0298-42e5-a4d0-8c34aaa60ec2', metadata={'source': './uploaded_pdfs/sample.txt'}, page_content='| Net cash from/(used in) investing activities                                              | 2,578,486                | (489,175)                    |\n| Cash flows from financing activities                                                      |                          |                              |\n| Proceeds from the issue of shares                                                         | -                        | 618,460                      |\n| Payment of share issue costs                                                              | -                        | (22,500)                     |\n| Payment of convertible note issue costs                                                   | (71,446)                 | -                            |\n| Proceeds from borrowings - convertible notes payable                                      | 500,000                

In [ ]:
ans=retriever.invoke("What is the total face value of the convertible note?")
print(ans)

Initialize the LLM

In [23]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

READER_MODEL_NAME = "HuggingFaceH4/zephyr-7b-beta"

# Configure quantization settings for 4-bit loading
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load the causal language model
model = AutoModelForCausalLM.from_pretrained(
    READER_MODEL_NAME, quantization_config=bnb_config
)

# Load the tokenizer
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

# Create a text-generation pipeline
llm = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=500,
)

/home/lahirum/Documents/Mirada/RnD/FastApi/test2/.venv/lib/python3.12/site-packages/torch/cuda/__init__.py:827: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Loading checkpoint shards: 100%|██████████| 8/8 [02:55<00:00, 21.90s/it]
Device set to use cpu


In [ ]:

# llm = Ollama(model="llama3.2", temperature=0.2)

In [18]:
chat_template = [
        {"role": "system", "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer."""},
        {"role": "user", "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}"""},
    ]

In [ ]:
# # Define a system prompt that tells the model how to use the retrieved context
# system_prompt = """You are an assistant for question-answering tasks.
# Use the following pieces of retrieved context to answer the question.
# If you don't know the answer, just say that you don't know.
# Use three sentences maximum and keep the answer concise.
# Context: {context}:"""


chat prompt template

In [20]:
from langchain_core.prompts import ChatPromptTemplate

# Define a chat prompt template composed of system and user messages
chat_template= ChatPromptTemplate([
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, do not give an answer.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
])


In [ ]:
# retrieved_docs_text = [
#     doc.page_content for doc in ans
# ]  # We only need the text of the documents
# context = "\nExtracted documents:\n"
# context += "".join(
#     [f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)]
# )

In [ ]:
question="What is the total face value of the convertible note?"

In [ ]:
docs = retriever.invoke(question)
context = "\n".join(d.page_content for d in docs)

RAG chain

In [21]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# Define a RAG chain using a pipeline of runnables
rag_chain=(
    {"context":retriever|(lambda docs:"\\n".join([d.page_content for d in docs])),   # Combine retrieved docs
     "question":RunnablePassthrough(),
     }
     |chat_template
     | (lambda chat_prompt: chat_prompt.to_string())         # Convert prompt object to string
     |(lambda text: llm(text)[0]["generated_text"].split("Answer:")[-1].strip())    # Generate and extract answer
     |StrOutputParser()
     
)

In [22]:
# Generate answer for a question
result = rag_chain.invoke("What is the total face value of the convertible note?")
print(result)

NameError: name 'llm' is not defined

In [20]:
# final_prompt = prompt_in_chat_format.format(
#     question="How to create a pipeline object?", context=context
# )

In [13]:
# answer = READER_LLM(final_prompt)[0]["generated_text"]
# print(answer)